In [16]:
import hourse_price_preprocessor
from stacking import MyStackingRegressor
import numpy as np
from pandas import DataFrame

from sklearn.model_selection import cross_val_score


In [10]:
import warnings
import copy
warnings.filterwarnings('ignore')

In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [17]:
trainset_file = "./house_price/train.csv"
test_file = "./house_price/test.csv"

X_train, X_test, y_train, test_id_idx = \
            hourse_price_preprocessor.get_train_test_split_dataset(trainset_file,test_file)

In [18]:
from xgboost import XGBRegressor
from lightgbm.sklearn import LGBMRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVR
from sklearn.neural_network import MLPRegressor

In [19]:
estimator1 = XGBRegressor(max_depth=3, learning_rate=0.5, n_estimators=50)
estimator2 = LGBMRegressor(max_depth=2, learning_rate=0.5, n_estimators=50)
estimator3 = RandomForestRegressor(n_estimators=500, max_depth=3)
estimator4 = GaussianNB()
estimator5 = LinearSVR()
estimator6 = MLPRegressor(hidden_layer_sizes=(512,256, 32))

In [20]:
base_estimators = [estimator1, estimator2, estimator3, estimator4, estimator5, estimator6]

In [21]:
meta_estimator = XGBRegressor(max_depth=3, learning_rate=0.5, n_estimators=50)

In [22]:
stack_regressor = MyStackingRegressor(meta_estimator, base_estimators)

In [27]:
stack_regressor.fit(X_train, y_train)

(292, 6)


MyStackingRegressor(base_estimators=[XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
...e=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)],
          feature_weights=None,
          meta_estimator=XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
          test_ratio=0.2)

In [31]:
cross_val_score(stack_regressor, X_train, y_train, scoring="mean_squared_error" , cv=5).mean()

-1209094010.8762403

In [34]:

for estimator in base_estimators:
    stack_regressor = MyStackingRegressor(copy.copy(estimator), base_estimators)
    stack_regressor.fit(X_train, y_train)
    result = cross_val_score(stack_regressor, X_train, y_train, scoring="mean_squared_error" , cv=5).mean()
    print(result)

-1437656288.675879
-1500632269.2301717
-942747658.5541519
-3542119858.521918
-1274039668.9304452
-1812187239.4012923


In [35]:
for estimator in base_estimators:
    general_estimator = copy.copy(estimator)
    general_estimator.fit(X_train, y_train)
    result = cross_val_score(general_estimator, X_train, y_train, scoring="mean_squared_error" , cv=5).mean()
    del general_estimator
    print(result)

-963969338.650704
-1141626511.0051265
-1800517249.3279395
-2113784463.8692105
-36887497534.32105
-1199706745.220332


In [40]:
general_estimator = copy.copy(base_estimators[0])
general_estimator.fit(X_train, y_train)
general_estimator.predict(X_test)

array([128621.04 , 155807.97 , 197459.06 , ..., 130143.836, 114837.03 ,
       260349.64 ], dtype=float32)

In [44]:
def make_submission(estimator, X_test, test_id_idx, filename, column_names):
    id_value=test_id_idx
    sales_price = estimator.predict(X_test)
    result = np.vstack([id_value, sales_price]).T
    submission_df =DataFrame(result, columns=column_names).set_index(column_names[0])
    submission_df.index = submission_df.index.astype(int)
    submission_df.to_csv(filename)


In [45]:
make_submission(general_estimator, X_test, test_id_idx)

In [48]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f house_price_result_search.csv -m "Message"

Successfully submitted to House Prices: Advanced Regression Techniques

In [50]:
general_estimator = copy.copy(base_estimators[2])
stack_regressor = MyStackingRegressor(general_estimator, base_estimators)
stack_regressor.fit(X_train, y_train)

MyStackingRegressor(base_estimators=[XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.5, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=50,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
...e=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)],
          feature_weights=None,
          meta_estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
          test_ratio=0.2)

In [51]:
make_submission(stack_regressor, X_test, test_id_idx)

In [53]:
!kaggle competitions submit -c house-prices-advanced-regression-techniques -f house_price_result_search.csv -m "Message"

Successfully submitted to House Prices: Advanced Regression Techniques

## Classifier problems

In [3]:
from xgboost import XGBClassifier
from lightgbm.sklearn import LGBMClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from stacking import MyStackingClassifier

In [4]:
X = np.load("./tatanic_X_train.npy")
y = np.load("./tatanic_y_train.npy")

In [13]:
estimator1 = XGBClassifier(max_depth=3, learning_rate=0.5, n_estimators=50)
estimator2 = LGBMClassifier(max_depth=2, learning_rate=0.5, n_estimators=50)
estimator3 = RandomForestClassifier(n_estimators=500, max_depth=3)
estimator4 = LinearSVC()
estimator5 = MLPClassifier(hidden_layer_sizes=(512,256, 32))

In [14]:
base_estimators = [estimator1, estimator2, estimator3, estimator4, estimator5]

In [22]:
for estimator in base_estimators:
    stack_classifier = MyStackingClassifier(copy.copy(estimator), base_estimators)
    result = cross_val_score(stack_classifier, X, y, scoring="accuracy" , cv=5).mean()
    print(result)

0.8290611312131023
0.8211769186821556
0.8268075922046595
0.8223259061766013
0.8087665841427031


In [23]:
for estimator in base_estimators:
    general_estimator = copy.copy(estimator)
    result = cross_val_score(general_estimator, X, y, scoring="accuracy" , cv=5).mean()
    del general_estimator
    print(result)

0.840290738272075
0.833561861232781
0.8087729321399099
0.8313083222243381
0.831321018218752


In [26]:
X_test = np.load("tatanic_test.npy")

In [32]:
idx = list(range(892, 1310))

In [73]:
general_estimator = copy.copy(base_estimators[5])
general_estimator.fit(X, y)

IndexError: list index out of range

In [69]:
column_names = ["PassengerId", "Survived"]

In [70]:
def make_submission(estimator, X_test, test_id_idx, filename, column_names):
    id_value=test_id_idx
    sales_price = np.array(estimator.predict(X_test), dtype=int)
    result = np.vstack([id_value, sales_price]).T
    submission_df =DataFrame(result, columns=column_names).set_index(column_names[0])
    submission_df.index = submission_df.index.astype(int)
    submission_df.to_csv(filename)


In [71]:
make_submission(general_estimator, X_test, idx, "titanic_submssion.csv", column_names)

In [72]:
!kaggle competitions submit -c titanic -f titanic_submssion.csv -m "Message"

Successfully submitted to Titanic: Machine Learning from Disaster

In [88]:
general_estimator = copy.copy(base_estimators[4])
stack_estimator = MyStackingClassifier(general_estimator, base_estimators)
result = stack_estimator.fit(X, y)
make_submission(result, X_test, idx, "titanic_submssion.csv", column_names)

In [89]:
!kaggle competitions submit -c titanic -f titanic_submssion.csv -m "Message"

Successfully submitted to Titanic: Machine Learning from Disaster